<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/timesformer/TimeSformer-huggingface-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 7.6 MB/s eta 0:00:00


In [4]:
import av # 동영상 파일을 처리하기 위한 라이브러리.
import torch # PyTorch 라이브러리로, 모델 학습 및 추론에 사용.
import numpy as np # 배열 및 수학적 연산을 위한 라이브러리.

from transformers import AutoImageProcessor, TimesformerForVideoClassification # Hugging Face Transformers 라이브러리로, 여기서는 Timesformer 모델을 로드하고 사용.
from huggingface_hub import hf_hub_download # Hugging Face Hub에서 데이터를 다운로드하기 위한 함수.

np.random.seed(0)

# PyAV를 사용하여 지정된 인덱스의 프레임을 읽어오는 함수입니다.
# container는 PyAV의 동영상 컨테이너 객체이고, indices는 읽어올 프레임 인덱스 리스트입니다.
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

# sample 8 frames
# sample_frame_indices(clip_len, frame_sample_rate, seg_len): 주어진 동영상에서 특정 프레임 인덱스를 샘플링하는 함수입니다.
# clip_len은 샘플링할 총 프레임 수
# frame_sample_rate는 샘플링할 프레임의 간격
# seg_len은 동영상의 총 프레임 수입니다.
indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

# 이미지 전처리 및 모델 로드
image_processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")

inputs = image_processor(list(video), return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# model predicts one of the 400 Kinetics-400 classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

preprocessor_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

eating spaghetti
